# Run Functions to Add Information to Projects

In [1]:
import numpy as np
import pandas as pd
from siuba import *

from shared_utils import geography_utils
from dla_utils import _dla_utils

from calitp import to_snakecase


import utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


ModuleNotFoundError: No module named 'calitp'

In [2]:
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', None)

## Read in Data and function development

In [3]:
df = utils.read_data_all()
    
    ## function that adds known agency name to df 
df = utils.identify_agency(df, 'summary_recipient_defined_text_field_1_value')

In [4]:
df.sample(3)

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
1226,2022-09-01,YS30,Highway Safety Improvement Program (HSIP),2152252,0819000045S,IN RIVERSIDE COUNTY NEAR MORENO VALLEY ON INTERSTATE 215 FROM 0.2 MILE NORTH OF ALESSANDRO BOULEVARD OVERCROSSING TO 0.2 MILE SOUTH OF EUCALYPTUS AVE,65.00,Cong Dist 41,17.00,Construction Engineering,"1,079,600.00",S SCAG,NaN,California,8.00,Riverside County,NaN,NaN
794,2022-09-20,Y001,National Highway Performance Program (NHPP),P070145,0315000082S,"IN AND NEAR MARYSVILLE, FROM SOUTH OF 14TH STREET TO NORTH OF CEMETERY ROAD. ROADWAY REHABILITATION AND OPERATIONAL IMPROVEMENTS INCLUDING TURN POCKE",115.00,Cong Dist 3,16.00,Right of Way,"2,000,000.00",S SACOG,NaN,California,3.00,Yuba County,NaN,NaN
847,2022-07-06,Y001,National Highway Performance Program (NHPP),P118068,0716000083S,"IN THE CITY OF LOS ANGELES, FROM TOPANGA CANYON BOULEVARD TO SEPULVEDA BOULEVARD. CONSTRUCT AND INSTALL STORMWATER TREATMENT BEST MANAGEMENT PRACTIC",37.00,Cong Dist 30,42.00,Training,488.23,S SCAG,NaN,California,7.00,Los Angeles County,NaN,NaN


## Test & Export

In [ ]:
GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

In [ ]:
###test agg. PASS df with title column, concat and 552 rows
agg = utils.get_clean_data(full_or_agg = 'agg')

In [ ]:
sorted(list(agg.project_title_new))

[' Added Roadway Capacity in Marin County',
 ' Added Roadway Capacity in Orange County',
 ' Bike Lanes in Alameda',
 ' Bike Share Program in Metropolitan Transportation Commission',
 ' Bike Share Program in Windsor',
 ' Bridge Preventive Maintenance in Humboldt County',
 ' Bridge Preventive Maintenance in Pleasanton',
 ' Bridge Rehabilitation in Chico',
 ' Bridge Rehabilitation in Contra Costa County',
 ' Bridge Rehabilitation in Los Angeles County',
 ' Bridge Rehabilitation in Monterey County',
 ' Bridge Rehabilitation in Palm Springs',
 ' Bridge Rehabilitation in Sacramento County',
 ' Bridge Rehabilitation in San Benito County',
 ' Bridge Rehabilitation in San Diego',
 ' Bridge Rehabilitation in San Francisco County',
 ' Bridge Rehabilitation in San Francisco County',
 ' Bridge Rehabilitation in San Joaquin County',
 ' Bridge Rehabilitation in San Joaquin County',
 ' Bridge Rehabilitation in San Luis Obispo County',
 ' Bridge Rehabilitation in Sierra County',
 ' Bridge Rehabilitatio

In [ ]:
agg.sample(5)

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,project_title,obligations_amount,congressional_district,district,county_code,county_name,implementing_agency_locode,rtpa_name,mpo_name,project_title_new
101,2022-04-29,P051031,California,S SACOG,Y001,National Highway Performance Program (NHPP),0316000113S,15.0,Preliminary Engineering,SR 51 FROM J STREET TO ARDEN WAY IN SACRAMENTO SR 51 CORRIDOR IMPROVEMENTS,"3,007,542.84",Cong Dist 51,11.0,73.0,San Diego County,nan,None,None,Preliminary Engineering Projects in San Diego County
211,2022-06-12,0055156,California,S MCAG,Y001,National Highway Performance Program (NHPP),1016000046S,5.0 | 17.0 | 42.0,4R - Maintenance Resurfacing | Construction Engineering | Training,"MERCED COUNTY 0.4 MILE NORTH OF THE FRESNO COUNTY LINE AT THE JOHN ""CHUCK"" ERRECA SAFETY ROADSIDE REST AREA REPLACE SAFETY ROADSIDE REST AREA (SRRA)","31,405,894.22",Cong Dist 16,10.0,47.0,Merced County,nan,None,None,Maintenance Resurfacing in Merced County
483,2022-09-13,5451023,Mission Viejo,L5451SCAG,Y230,Surface Transportation Block Grant (STBG) Program,12931559L,13.0 | 17.0,Bridge Rehabilitation - Added Capacity | Construction Engineering,LA PAZ ROAD: MUIRLANDS BLVD. TO CRISANTA DR. WIDEN TWO OVERHEADS OVER BNSF,"5,420,508.00",Cong Dist 45,12.0,59.0,Orange County,5451.0,Orange County Transportation Authority,Southern California Association Of Governments,Widen Road in Mission Viejo
478,2022-09-12,P204006,California,S KCOG,YS30,Highway Safety Improvement Program (HSIP),0617000297S,17.0 | 21.0,Construction Engineering | Safety,KERN COUNTY IN BAKERSFIELD AT 8TH STREET. INSTALLATION OF HIGH INTENSITY ACTIVATED CROSSWALK.,"1,818,500.00",Cong Dist 21,9.0,29.0,Kern County,nan,None,None,Safety Improvements in Kern County
181,2022-05-18,5378049,Palmdale,L5378SCAG,Y230,Surface Transportation Block Grant (STBG) Program,0722000002L,1.0,New Construction Roadway,"AVE R BETWEEN SIERRA HIGHWAY AND 25TH STREET. AVE R COMPLETE STREET WITH SIDEWALKS GAP CLOSURES, BIKE LANES, ADA RAMPS AND ENHANCED CROSSWALKS","3,200,541.00",Cong Dist 25,7.0,37.0,Los Angeles County,5378.0,Los Angeles County Metropolitan Transportation Auth.,Southern California Association Of Governments,Complete Streets in Palmdale


In [ ]:
agg = utils.title_column_names(agg)

In [ ]:
# agg.to_csv(f"{GCS_FILE_PATH}/FMIS_projects_agg.csv")

In [ ]:
###test full. PASS title, no concat and 1241 rows
full = utils.get_clean_data(full_or_agg = 'full')

In [ ]:
full = utils.title_column_names(full)

In [ ]:
# full.to_csv(f"{GCS_FILE_PATH}/FMIS_projects_all.csv")